In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
#links
estados = ['https://www.conicyt.cl/blog/category/concursos/abiertos/', 'https://www.conicyt.cl/blog/category/concursos/en-evaluacion/', 'https://www.conicyt.cl/blog/category/concursos/adjudicados/', 'https://www.conicyt.cl/blog/category/concursos/proximos/']

def links_proyectos(estados_chile):

    links = []
    for estado in estados:

        parserT = parser(estado)

        #Extrae y organiza lista de paginacion
        paginacion = parserT.xpath('//div[@class="pagenavi"]/ul//a/@href')
        
        urls = []
        pagina = 'https://www.conicyt.cl/blog/category/concursos/adjudicados/'
        if len(paginacion) != 0:
            paginacion.pop()
            paginacion.append('?pag=1')
            for link in paginacion:
                urls.append(pagina+link)
        else:
            urls.append(estado)
        
        

        #Extrae links de cada pag
        
        for url in urls:
            parser_url = parser(url)
            links_pagina = parser_url.xpath('//div[@id="contenido_doble"]//h2/a/@href')
            links += links_pagina
        
        
        for link in links:
            if link.startswith('https://www.conicyt.cl/becasconicyt'):
                links.remove(link)


    return links

links = links_proyectos(estados)



In [ ]:
#ano
parser_proy = parser(links[0])
links[0]

In [ ]:
### avance chile

descripciones = []
publicos = []
bitacoras = []
descripcion = ''
p_objetivo = ''
bitacora = ''
titulo = ''
for link in links[0:5]:

    driver = webdriver.Firefox()
    driver.get(link)

    #Saca Titulo
        
    try:
        titulo = driver.find_element_by_xpath('//h2[@id="title-concurso"]').text

    except:
        
        try:
            if len(titulo) == 0:
                titulo = driver.find_element_by_xpath('//h2[@class="title_content"]').text
        except:        
            titulo = ''


    ### Saca el formato descripción

    try:
        boton_descripcion = driver.find_element_by_xpath('//div[contains(text(), "presentación")]')
            
    except:
        try:
            boton_descripcion = driver.find_element_by_xpath('//li[contains(text(), "Presentación")]')
        except:
            try:
                boton_descripcion = driver.find_element_by_xpath('//div[contains(@class, "accordion-Presentacin")]')
            except:
                descripcion = ''
                continue
                
                
    boton_descripcion.click()

    try:
        descripcion = driver.find_element_by_xpath('.//div[@id="content-1"]').text

    except:
        try:
            descripcion = driver.find_element_by_xpath('.//div[@id="pane-0-0"]').text
        except:
            descripcion = driver.find_element_by_xpath('.//div[@class="arconix-accordion-content"]').text



    ### Saca el publico objetivo

   
    try:
        boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "público")]')
    except:
        try:
            boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "Público")]')
        except:
            try:
                boton_p_objetivo = driver.find_element_by_xpath('//ul//li[contains(text(), "Público")]') 
            except:
                p_objetivo = ''
                continue
                
            
    boton_p_objetivo.click()
    try:
        p_objetivo = driver.find_element_by_xpath('.//div[@id="content-2"]').text
    except:
        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text

    
    ### Saca Bitacoras
    try:
        boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitacora")]')
    except:
        try:
            boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitácora")]')
        except:
            try:
                boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitácora")]')
            except:
                try:
                    boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitacora")]')
                except:
                    try:
                        boton_bitacora = driver.find_element_by_xpath('//ul//li[contains(text(), "bitacora")]') 
                    except:
                        try:
                            boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitácora")]')
                        except:
                            bitacora = ''
                            continue
                            
                    
            
    boton_bitacora.click()
    
    try:
        bitacora = driver.find_element_by_xpath('.//div[@id="content-3"]').text
    except:
        
        try:
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
        except:
            bitacora = ''

    ### Extrae los links
        try:
            url = driver.find_elements_by_xpath('//div[@class="collapse show"]/div/ul//li/a')

            urls = []
            for r in url:
                url2 = r.get_attribute('href')
                urls.append(url2)



            if len(urls) == 0:
                url = driver.find_elements_by_xpath('//div[@class="filtros"]/ul//li/a')

                urls = []
                for r in url:
                    url2 = r.get_attribute('href')
                    urls.append(url2)

            urls2 = ''
            for url2 in urls:
                urls2 = urls2 + ', ' + url2
        except:
            urls2 = ''

                
    driver.quit()
    sleep(random.randrange(3, 5))


    print('1')
    print(descripcion)
    print('2')
    print(p_objetivo)
    print('3')
    print(bitacora)
    print('4')
    print(titulo)
    print('5')
    print(urls2.lstrip())


In [ ]:
random.randrange(3, 5)

In [237]:


for link in links[0:20]:
    print(link)

    driver = webdriver.Firefox()
    driver.get(link)
    try:
        url = driver.find_elements_by_xpath('//div[@class="collapse show"]/div/ul//li/a')

        urls = []
        for r in url:
            url2 = r.get_attribute('href')
            urls.append(url2)



        if len(urls) == 0:
            url = driver.find_elements_by_xpath('//div[@class="filtros"]/ul//li/a')

            urls = []
            for r in url:
                url2 = r.get_attribute('href')
                urls.append(url2)

        urls2 = ''
        for url2 in urls:
            urls2 = urls2 + ', ' + url2
    except:
        print("Error")


    print(urls2.lstrip(', '))
    driver.quit()



https://www.conicyt.cl/informacioncientifica/2017/06/13/convocatoria-para-indexacon-de-revistas-cientificas-en-scielo-chile/

https://www.conicyt.cl/fondef/2019/11/20/i-concurso-idea-en-dos-etapas-tematico-en-inocuidad-y-calidad-alimentaria-segunda-etapa/
https://www.conicyt.cl/fondef/files/2019/11/Bases-I-Concurso-IDeA-dos-etapas-Tematico-en-Inocuidad-y-Calidad-Alimentaria.pdf, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-ETAPA-2-PRESENTACIoN.ppt, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-ETAPA-2-Formulario-de-postulacion.docx, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-ETAPA-2-CARTA-DE-COMPROMISO-INSTITUCION-BENEFICIARIA.doc, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-CARTA-COMPROMISO-ENTIDAD-ASOCIADA.doc, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-DECLARACIoN-JURADA-SIMPLE.doc, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-DECLARACIoN-SIMPLE-DUPLICIDAD.doc, https://www.conicyt.cl/fondef/files/2019/09/ID17AL2-ETAPA-2-CARTA-EVALUADORES